In [1]:
from bs4 import BeautifulSoup
from pandas import DataFrame
from time import sleep
import urllib.request
import pandas as pd
import numpy as np
import requests
import datetime
import requests
import os
import io
import re

In [2]:
location='sfbay'

In [3]:
url = f'https://{location}.craigslist.org/d/cars-trucks-by-owner/search/cto'
Links=[]
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
total_posts=int(soup.find("span", {"class": "total"}).text)
fraction=total_posts%120

In [4]:

final_links=[]

for i in range(0,total_posts-fraction,120):
    url = f'https://{location}.craigslist.org/d/cars-trucks-by-owner/search/cto?s={i}'
    Links=[]
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
        Links.append(link.get('href'))
    
    for i in range(9, 249):
        if i%2==0:
            final_links.append(Links[i])
            

In [5]:
len(final_links)

3000

In [6]:
spec_list=[]
title_list=[]
price_list=[]

for lin in final_links:
    try:
        page = requests.get(lin)
        soup = BeautifulSoup(page.text, "html.parser")
        spec_list.append(soup.find_all("p", {"class": "attrgroup"})[1].text.strip('\n').replace("\n"," ").split(" "))
        title_list.append(soup.find_all("p", {"class": "attrgroup"})[0].text.strip('\n'))
        price_list.append(str(soup.find("span", {"class": "price"})).strip('$').replace(',',""))
    except:
        continue


In [7]:
df_=pd.DataFrame(columns=['year:','make:','model:','trim:','VIN:','condition:','odometer:',
                          'cylinders:','color:','transmission:',
                          'type:','status:','drive:','fuel:','price:'])


In [8]:
for i,title in enumerate(title_list):
    if(len(title.split())>=4 and not(title.split()[1].isnumeric())):
            df_.loc[f'{i}','year:']=title.split()[0]
            df_.loc[f'{i}','make:']=title.split()[1]
            df_.loc[f'{i}','model:']=title.split()[2]
            df_.loc[f'{i}','trim:']=title.split()[3]
    if(len(title.split())==3 and not(title.split()[1].isnumeric())):
            df_.loc[f'{i}','year:']=title.split()[0]
            df_.loc[f'{i}','make:']=title.split()[1]
            df_.loc[f'{i}','model:']=title.split()[2]
            df_.loc[f'{i}','trim:']= np.nan
    if(len(title.split())==4 and title.split()[1].isnumeric()):
            df_.loc[f'{i}','year:']=title.split()[0]
            df_.loc[f'{i}','make:']=title.split()[2]
            df_.loc[f'{i}','model:']=title.split()[3]
            df_.loc[f'{i}','trim:']= np.nan
    if(len(title.split())>=5 and title.split()[1].isnumeric()):
            df_.loc[f'{i}','year:']=title.split()[0]
            df_.loc[f'{i}','make:']=title.split()[2]
            df_.loc[f'{i}','model:']=title.split()[3]
            df_.loc[f'{i}','trim:']= title.split()[4]
            

In [9]:
for i,temp in enumerate(spec_list):
    for word in temp:
        if(word=='VIN:'):
            df_.loc[f'{i}','VIN:']=temp[temp.index(word)+1]    
        if(word=='condition:'):
             df_.loc[f'{i}','condition:']=temp[temp.index(word)+1] 
        if(word=='odometer:'):
            df_.loc[f'{i}','odometer:']=temp[temp.index(word)+1]
        if(word=='cylinders:'):
             df_.loc[f'{i}','cylinders:']=temp[temp.index(word)+1]
        if(word=='color:'):
             df_.loc[f'{i}','color:']=temp[temp.index(word)+1]
        if(word=='transmission:'):
             df_.loc[f'{i}','transmission:']=temp[temp.index(word)+1]
        if(word=='type:'):
             df_.loc[f'{i}','type:']=temp[temp.index(word)+1]
        if(word=='status:'):
             df_.loc[f'{i}','status:']=temp[temp.index(word)+1]
        if(word=='drive:'):
             df_.loc[f'{i}','drive:']=temp[temp.index(word)+1] 
        if(word=='fuel:'):
             df_.loc[f'{i}','fuel:']=temp[temp.index(word)+1]
                          

In [10]:
for i,p in enumerate(price_list):
    df_.loc[f'{i}','price:']=p

In [11]:
df_['price:']=df_['price:'].apply(lambda x: x[21:-7])
df_['price:'] = pd.to_numeric(df_['price:'])
df_['year:'] = pd.to_numeric(df_['year:'])
df_['odometer:'] = pd.to_numeric(df_['odometer:'])

In [12]:
df_.shape

(2997, 15)

In [13]:
df_final=df_.dropna(axis=0, thresh=2, subset=['year:','make:','model:','trim:'], inplace=False)

In [14]:
df_final.shape

(2913, 15)

In [15]:
ct = datetime.datetime.now() 
name=str(ct)
date=name[0:-7].replace(":",'_')
date.replace(" ",'_')
df_final.to_csv(rf'C:\Users\Silmi\Desktop\UsedCar-PriceEstimator\data\craigslist_{date}.csv')